# Excercise 1 Context

1. Sketch a contextsensitive Application

2. Categorize it according to Schilit (explain why)

3. Categorize all relevant context regarding category of context, passive/active and primary/secondary

4. Modify a. to be in another class according to Schilit (Tip: You may modify the previous answer if this does not work)

# Excercise 2 Sensors

WebApp

# Excercise 3 Calculating features

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from influxdb import InfluxDBClient, DataFrameClient

## 3.1 Load Data

In [2]:
# all data in browser database
influxClient = DataFrameClient(host='css20.dmz.teco.edu', port=80, database='browser',username="user", password="pass")
data = influxClient.query("select * FROM devicemotion")["devicemotion"]

In [23]:
# own recorded data
influxClient = DataFrameClient(host='css20.dmz.teco.edu', port=80, database='motion_data',username="user", password="pass")
own_data = influxClient.query("select * FROM devicemotion")["devicemotion"]

In [29]:
own_data

,acceleration.x,acceleration.y,acceleration.z,accelerationIncludingGravity.x,accelerationIncludingGravity.y,accelerationIncludingGravity.z,label,mobile,rotationRate.alpha,rotationRate.beta,rotationRate.gamma,subject,useragent
2020-10-04 08:07:37.441600+00:00,-0.200000,0.200000,0.200000,-2.200000,8.600000,4.700000,testing,Samsung,-15.200000,-17.300000,6.00000,1b6cc,Mozilla/5.0 (Linux; Android 9; SM-G950F) Apple...
2020-10-04 08:07:37.463000+00:00,-0.100000,0.200000,0.100000,-2.100000,8.600000,4.700000,testing,Samsung,-9.800000,-18.900000,7.40000,1b6cc,Mozilla/5.0 (Linux; Android 9; SM-G950F) Apple...
2020-10-04 08:07:37.478000+00:00,0.200000,0.000000,0.100000,-1.800000,8.400000,4.700000,testing,Samsung,-5.300000,-5.000000,8.10000,1b6cc,Mozilla/5.0 (Linux; Android 9; SM-G950F) Apple...
2020-10-04 08:07:37.490100+00:00,0.400000,-0.200000,0.200000,-1.500000,8.200000,4.900000,testing,Samsung,-5.500000,-6.000000,8.00000,1b6cc,Mozilla/5.0 (Linux; Android 9; SM-G950F) Apple...
2020-10-04 08:07:37.507300+00:00,0.400000,-0.100000,0.000000,-1.500000,8.300000,4.700000,testing,Samsung,-4.200000,-6.400000,3.80000,1b6cc,Mozilla/5.0 (Linux; Android 9; SM-G950F) Apple...
2020-10-04 08:07:37.523100+00:00,0.400000,0.000000,0.100000,-1.500000,8.400000,4.800000,testing,Samsung,-2.300000,-0.900000,2.00000,1b6cc,Mozilla/5.0 (Linux; Android 9; SM-G950F) Apple...
2020-10-04 08:07:37.539500+00:00,0.100000,0.100000,0.000000,-1.800000,8.500000,4.700000,testing,Samsung,0.500000,-0.200000,-0.80000,1b6cc,Mozilla/5.0 (Linux; Android 9; SM-G950F) Apple...
2020-10-04 08:07:37.563000+00:00,0.000000,0.100000,0.000000,-2.000000,8.500000,4.700000,testing,Samsung,1.500000,-0.100000,-1.40000,1b6cc,Mozilla/5.0 (Linux; Android 9; SM-G950F) Apple...
2020-10-04 08:07:37.572900+00:00,-0.200000,0.100000,0.000000,-2.000000,8.500000,4.700000,testing,Samsung,1.200000,0.200000,-1.10000,1b6cc,Mozilla/5.0 (Linux; Android 9; SM-G950F) Apple...
2020-10-04 08:07:37.587500+00:00,-0.100000,0.100000,0.000000,-2.000000,8.500000,4.700000,testing,Samsung,0.400000,1.400000,-0.70000,1b6cc,Mozilla/5.0 (Linux; Android 9; SM-G950F) Apple...


In [3]:
data.columns

Index(['acceleration.x', 'acceleration.y', 'acceleration.z',
       'accelerationIncludingGravity.x', 'accelerationIncludingGravity.y',
       'accelerationIncludingGravity.z', 'label', 'mobile',
       'rotationRate.alpha', 'rotationRate.beta', 'rotationRate.gamma',
       'subject', 'useragent'],
      dtype='object')

In [4]:
data.head(2)

,acceleration.x,acceleration.y,acceleration.z,accelerationIncludingGravity.x,accelerationIncludingGravity.y,accelerationIncludingGravity.z,label,mobile,rotationRate.alpha,rotationRate.beta,rotationRate.gamma,subject,useragent
2020-05-18 15:38:31.330200+00:00,0.0514,-0.4303,1.4837,0.038307,5.439626,9.825804,testing,UnknownPhone,-13.115,-43.004999,0.488,10b5c,Mozilla/5.0 (Linux; Android 7.0; FRD-L19) Appl...
2020-05-18 15:38:31.348400+00:00,0.6009,-0.1249,1.0312,0.172383,5.477933,9.174581,testing,UnknownPhone,-2.623,-6.527000,-2.379,10b5c,Mozilla/5.0 (Linux; Android 7.0; FRD-L19) Appl...


In [5]:
data.shape

(346487, 13)

In [6]:
# Select time window
data = data.loc['2020-05-01':].dropna()
data.shape

(179951, 13)

## 3.2 Preprocessing

In [7]:
# Extract labels and subjects
labels = data['label'].unique()[1:] # drop "testing"-label
subjects = data['subject'].unique()

# Group data according to labels and subjects
grouped_data = {}
for subject in subjects:
    grouped_data[subject] = {}
    subject_data = data[data['subject'] == subject]
    for label in labels:
        grouped_data[subject][label] = subject_data[subject_data["label"] == label]

## 3.3 Windowing

In [8]:
def minmax(data):
    return np.max(data)-np.min(data)

In [9]:
# Prepare aggregation
numeric_columns = {}
for col, dtype in zip(data.columns, data.dtypes):
    if dtype == 'float64':
        numeric_columns[col] = ['max', 'mean', 'min',"std", "var",minmax]

In [10]:
aggregatedDict = {}
for subject in subjects:
    aggregatedDict[subject] = {}
    for label in labels:
        # check if part is empty
        if(not grouped_data[subject][label].empty):
            # use resample for frequency conversion 
            aggregatedDict[subject][label] = grouped_data[subject][label].resample('1s').agg(numeric_columns).dropna()

## 3.4 Data Transformation

In [11]:
# groups for Leave-One-Subject-Out-CV
groups = []

# Transform data into ungrouped and flat table
data = pd.DataFrame()
for subject in subjects:
    for label in labels:
        if label in aggregatedDict[subject]:
            curr_data = aggregatedDict[subject][label]
            curr_labels = len(curr_data)*[label]
            curr_data["label"] = curr_labels
            data = data.append(curr_data)
            groups.extend(len(curr_data)*[subject])
# append groups
data["group"] = groups        

# 3.5 Robust Outlier-Cleansing

In [12]:
# RMSE 3D_acceleration
minmax_3d_acc = np.sqrt(data["acceleration.x"]["minmax"]**2 + data["acceleration.y"]["minmax"]**2
                        + data["acceleration.z"]["minmax"]**2)

In [13]:
# Calculate Quantiles
Q1 = np.percentile(minmax_3d_acc, 25)  
Q3 = np.percentile(minmax_3d_acc, 75) 
# Calculate IQR
IQR = Q3 - Q1 
# Calculate bounds
low_lim = np.array(Q1 - 1.5 * IQR)
up_lim = np.array(Q3 + 1.5 * IQR)
# Get index of outliers
index = np.logical_and(minmax_3d_acc.values >= low_lim, minmax_3d_acc.values <= up_lim)

In [14]:
# Remove outliers
data = data.loc[index]

# 3.6 Visualization of recorded data



# Excercise 4 Creating a predictor

In [15]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.feature_selection import RFE

## 4.1 Train-Test Split

In [16]:
# Seed
np.random.seed(5050)

# Draw random Training-Data-Indices
number_subjects = len(subjects)
size = int(0.7*number_subjects)

# Extract Train/Test-Subjects
train_subjects = np.random.choice(subjects, size=size, replace=False)
test_subjects = list(set(subjects)-set(train_subjects))

# Setup train and test data
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for train_subject in train_subjects:
    train_data= train_data.append(data.loc[data["group"] == train_subject])
    
for test_subject in test_subjects:
    test_data= test_data.append(data.loc[data["group"] == test_subject])
    
# finalize train / test data
x_train = train_data.drop(["group", "label"], axis="columns")
x_test = test_data.drop(["group", "label"], axis="columns")
y_train = train_data["label"]
y_test = test_data["label"]

## 4.2 10-Fold-CV

In [17]:
# Seed
np.random.seed(1010)

# MLP
mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=100, early_stopping = True)
mlp_res = cross_validate(mlp, x_train, y_train, n_jobs=2, cv=10,
                         return_estimator=False, return_train_score=True)

# RF
rf = RandomForestClassifier(n_estimators=100, max_depth=3)
rf_res = cross_validate(rf, x_train, y_train, n_jobs=2, cv=10,
                         return_estimator=False, return_train_score=True)

print("MLP 10-Fold-CV-Accuracy: ", mlp_res["test_score"].mean())
print("Random Forest 10-Fold-CV-Accuracy: ", rf_res["test_score"].mean())

MLP 10-Fold-CV-Accuracy:  0.7497706182879782
Random Forest 10-Fold-CV-Accuracy:  0.712324053800438


## 4.3 Leave-One-Subject-Out-CV

In [18]:
# Seed
np.random.seed(1111)

logo = LeaveOneGroupOut()
logo.get_n_splits(groups=train_data["group"].values)

# MLP
mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=100, early_stopping = True)
mlp_res = cross_validate(mlp, x_train, y_train, cv=logo, n_jobs=2, groups = train_data["group"].values,
                         return_estimator=False, return_train_score=True)

# RF
rf = RandomForestClassifier(n_estimators=100, max_depth=3)
rf_res = cross_validate(rf, x_train, y_train, cv=logo, n_jobs=2, groups = train_data["group"].values,
                        return_estimator=False, return_train_score=True)

print("MLP LOGO-CV-Accuracy: ", mlp_res["test_score"].mean())
print("Random Forest LOGO-CV-Accuracy: ", rf_res["test_score"].mean())

MLP LOGO-CV-Accuracy:  0.7984543579116767
Random Forest LOGO-CV-Accuracy:  0.7948998995896946


## 4.4 Recursive Feature Elimination

In [19]:
# Seed
np.random.seed(1212)

rfe = RFE(rf, n_features_to_select=10, step=1, verbose=0)
rfe.fit(x_train,y_train)

# evaluate
selected_features = x_train.columns[rfe.support_]
x_train_new = x_train[selected_features]

rf_res_new = cross_validate(rf, x_train_new, y_train, n_jobs=2, cv=logo, groups = train_data["group"].values,
                         return_estimator=False, return_train_score=True)
print("Random Forest - RFE -  LOGO-CV-Accuracy: ", rf_res_new["test_score"].mean())

Random Forest - RFE -  LOGO-CV-Accuracy:  0.7950086836942393


## 4.5 Classifier with Bayes-Optimization

### 4.5.1. MLP

In [20]:
from hyperopt import Trials, STATUS_OK, tpe, fmin, hp
# set seeds
np.random.seed(4321)

# define space
mlp_space = {'hidden_layer_sizes' : hp.quniform('hidden_layer_sizes',5,50,1),
             'max_iter' : hp.quniform("max_iter",10,500,1)}
max_iters = 10

def mlp_opt(params):
    from sklearn.model_selection import LeaveOneGroupOut
    from sklearn.neural_network import MLPClassifier
    from sklearn.model_selection import cross_validate
    
    params["hidden_layer_sizes"] = int(params["hidden_layer_sizes"])
    params["max_iter"] = int(params["max_iter"])
    
    mlp = MLPClassifier(hidden_layer_sizes= params["hidden_layer_sizes"], max_iter= params["max_iter"])
    mlp = cross_validate(mlp, x_train_new, y_train, cv=10, n_jobs=2,
                         return_estimator=False, return_train_score=True)
    
    score = -1* mlp["test_score"].mean()
    return {'loss': score, 'status': STATUS_OK, 'model': None, 'params': params}

### optimization
trials = Trials()
best_mlp = fmin(mlp_opt, mlp_space, algo=tpe.suggest, max_evals= max_iters,trials=trials,
                rstate= np.random.RandomState(1234))
print('best: ')
### Extract params
best_MLP_params = trials.results[np.argmin([r['loss'] for r in trials.results])]['params']
print(best_MLP_params)
# Extract Model
MLP_model = MLPClassifier(hidden_layer_sizes = best_MLP_params["hidden_layer_sizes"], max_iter = best_MLP_params["max_iter"])
MLP_model.fit(x_train_new, y_train)

100%|███████████████████████████████████████████████| 10/10 [01:46<00:00,  9.71s/trial, best loss: -0.7721978938588261]
best: 
{'hidden_layer_sizes': 43, 'max_iter': 232}


C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (232) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(hidden_layer_sizes=43, max_iter=232)

### 4.5.2. Random Forest

In [21]:
from hyperopt import Trials, STATUS_OK, tpe, fmin, hp
# set seeds
np.random.seed(4321)

# define space
rf_space = {'n_estimators' : hp.quniform('n_estimators',5,2000,1),
             'max_depth' : hp.quniform("max_depth",1,3,1)}
max_iters = 10

def rf_opt(params):
    from sklearn.model_selection import LeaveOneGroupOut
    from sklearn.neural_network import MLPClassifier
    from sklearn.model_selection import cross_validate
    
    logo = LeaveOneGroupOut()
    logo.get_n_splits(groups=train_data["group"].values)
    
    params["n_estimators"] = int(params["n_estimators"])
    params["max_depth"] = int(params["max_depth"])
    
    rf = RandomForestClassifier(n_estimators=params["n_estimators"], max_depth=params["max_depth"])
    rf = cross_validate(rf, x_train_new, y_train, cv=logo, n_jobs=2, groups = train_data["group"].values,
                         return_estimator=False, return_train_score=True)
    
    score = -1* rf["test_score"].mean()
    return {'loss': score, 'status': STATUS_OK, 'model': None, 'params': params}

### optimization
trials = Trials()
best_rf = fmin(rf_opt, rf_space, algo=tpe.suggest, max_evals= max_iters,trials=trials,
                rstate= np.random.RandomState(1234))
print('best: ')
### Extract params
best_RF_params = trials.results[np.argmin([r['loss'] for r in trials.results])]['params']
print(best_RF_params)
# Extract Model
RF_model = RandomForestClassifier(n_estimators = best_RF_params["n_estimators"], max_depth = best_RF_params["max_depth"])
RF_model.fit(x_train_new, y_train)

100%|███████████████████████████████████████████████| 10/10 [05:14<00:00, 27.26s/trial, best loss: -0.7958812840432795]
best: 
{'max_depth': 3, 'n_estimators': 612}


RandomForestClassifier(max_depth=3, n_estimators=612)

## 4.6 Evaluate on Test-Data

In [22]:
RF_model.score(x_test[selected_features],y_test)

0.7064676616915423